In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow import keras
os.environ['KAGGLE_USERNAME'] = "aminnajaf"
os.environ['KAGGLE_KEY'] = "fcbc4abc30fb80fcedc3e58ce65eed9c"

In [2]:
!kaggle datasets download -d nipunarora8/age-gender-and-ethnicity-face-data-csv

age-gender-and-ethnicity-face-data-csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
from zipfile import ZipFile

In [4]:
with ZipFile('/content/age-gender-and-ethnicity-face-data-csv.zip', 'r') as zipObj:
   zipObj.extractall()

In [5]:
data = pd.read_csv('/content/age_gender.csv')

In [6]:
data.head()

,age,ethnicity,gender,img_name,pixels
0,1,2,0,20161219203650636.jpg.chip.jpg,129 128 128 126 127 130 133 135 139 142 145 14...
1,1,2,0,20161219222752047.jpg.chip.jpg,164 74 111 168 169 171 175 182 184 188 193 199...
2,1,2,0,20161219222832191.jpg.chip.jpg,67 70 71 70 69 67 70 79 90 103 116 132 145 155...
3,1,2,0,20161220144911423.jpg.chip.jpg,193 197 198 200 199 200 202 203 204 205 208 21...
4,1,2,0,20161220144914327.jpg.chip.jpg,202 205 209 210 209 209 210 211 212 214 218 21...


In [7]:
data.dtypes

age           int64
ethnicity     int64
gender        int64
img_name     object
pixels       object
dtype: object

In [8]:
data['ethnicity'].value_counts()

0    10078
1     4526
3     3975
2     3434
4     1692
Name: ethnicity, dtype: int64

In [9]:
data['gender'].value_counts()

0    12391
1    11314
Name: gender, dtype: int64

In [10]:
data = data.drop('ethnicity', axis=1)

In [11]:
age = data['age'].values
gender = data['gender'].values

In [12]:
pixels = data['pixels']

In [13]:
from tqdm import tqdm
def get_img(images):
  imgs = []
  for pixels in tqdm(images):
    pixels = pixels.split()
    img = [int(i) for i in pixels]
    imgs.append(np.array(img)/255.0)
  imgs = np.array(imgs).reshape(-1, 48, 48, 1)
  return imgs

In [14]:
imgs = get_img(pixels)

100%|██████████| 23705/23705 [00:14<00:00, 1628.95it/s]


In [15]:
img = keras.layers.Input(shape=(48, 48, 1))
x = keras.layers.Conv2D(filters=32, kernel_size=5, activation='relu', padding='same')(img)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPooling2D(2,2)(x)


x1 = keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(x)
x1 = keras.layers.BatchNormalization()(x1)
x1 = keras.layers.MaxPooling2D(2,2)(x1)
x1 = keras.layers.Flatten()(x1)
x1 = keras.layers.Dense(32, activation='relu')(x1)
x1 = keras.layers.Dropout(0.2)(x1)
out1 = keras.layers.Dense(1, name='out1')(x1)

x2 = keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(x)
x2 = keras.layers.BatchNormalization()(x2)
x2 = keras.layers.MaxPooling2D(2,2)(x2)
x2 = keras.layers.Flatten()(x2)
x2 = keras.layers.Dense(32, activation='relu')(x2)
x2 = keras.layers.Dropout(0.2)(x2)
out2 = keras.layers.Dense(1, activation='sigmoid', name='out2')(x2)

model = keras.models.Model(img, [out1, out2])

model.compile(loss={'out1':'mae', 'out2':'binary_crossentropy'}, optimizer=keras.optimizers.Adam(0.0001), loss_weights={'out1':2, 'out2':1}, metrics={'out2':['accuracy']})

In [16]:
class myCallback(keras.callbacks.Callback):
  best = 999
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('val_loss') < self.best:
      self.best = logs.get('val_loss')
      model.save('best_model.hdf5')
callback = myCallback()

In [17]:
history = model.fit(x=imgs, y=[age, gender], batch_size=32, epochs=50, validation_split=0.2, callbacks=[callback])

Epoch 1/50
593/593 [==============================] - 12s 15ms/step - loss: 19.6137 - out1_loss: 9.5949 - out2_loss: 0.4239 - out2_accuracy: 0.7969 - val_loss: 80.3562 - val_out1_loss: 39.9640 - val_out2_loss: 0.4282 - val_out2_accuracy: 0.8000
Epoch 2/50
593/593 [==============================] - 8s 13ms/step - loss: 13.4053 - out1_loss: 6.5755 - out2_loss: 0.2542 - out2_accuracy: 0.8873 - val_loss: 53.4277 - val_out1_loss: 26.5258 - val_out2_loss: 0.3762 - val_out2_accuracy: 0.8340
Epoch 3/50
593/593 [==============================] - 8s 13ms/step - loss: 12.2596 - out1_loss: 6.0268 - out2_loss: 0.2060 - out2_accuracy: 0.9123 - val_loss: 46.7132 - val_out1_loss: 23.1476 - val_out2_loss: 0.4179 - val_out2_accuracy: 0.8243
Epoch 4/50
593/593 [==============================] - 8s 14ms/step - loss: 11.8158 - out1_loss: 5.8189 - out2_loss: 0.1779 - out2_accuracy: 0.9258 - val_loss: 46.6137 - val_out1_loss: 23.1315 - val_out2_loss: 0.3506 - val_out2_accuracy: 0.8574
Epoch 5/50
593/593 [===

In [20]:
gender

array([0, 0, 0, ..., 1, 1, 1])